# Comprehensive Spike and Burst Analysis with Visibility Graphs
This notebook provides a detailed workflow for analyzing electrophysiological ABF recordings. It goes through the following steps:

- Loading ABF files and concatenating sweeps
- Spike detection
- Burst detection
- Burst classification
- Saving bursts to CSV
- Visibility graph creation for each burst
- 2D and 3D embeddings of the burst graphs
- Visualization of spikes, bursts, and graphs
- Saving nodes and edges data to CSV

Each section includes detailed explanations of the code functionality and rationale behind each step.

## Reference Screenshot
This screenshot shows an example of the spike and burst analysis:

![Spike and Burst Analysis](Captura_de_pantalla_2025-10-02_080635.png)

> Make sure the image file is uploaded to the same folder as the notebook, or adjust the path accordingly.

In [ ]:
import pyabf  # Reads ABF electrophysiology files
import numpy as np  # Numerical operations
import pandas as pd  # DataFrames and CSV handling
import matplotlib.pyplot as plt  # Plotting
import networkx as nx  # Graph creation and analysis
from scipy.signal import find_peaks  # Spike detection
from sklearn.decomposition import TruncatedSVD  # Embedding
%matplotlib widget

In [ ]:
file_path = "bursting/cell89basal.abf"
abf = pyabf.ABF(file_path)

# Concatenate all sweeps
signal = np.concatenate([abf.setSweep(i) or abf.sweepY for i in range(abf.sweepCount)])
dt = 1.0 / abf.dataRate
time = np.arange(len(signal)) * dt

print(f"File: {file_path} | sweeps: {abf.sweepCount} | total duration: {time[-1]:.2f} s")

In [ ]:
threshold = -35  # mV
spike_indices, _ = find_peaks(signal, height=threshold)
spike_times = time[spike_indices]

In [ ]:
isi = np.diff(spike_times)
burst_threshold = 0.3
bursts = []
current_burst = [0]

for i in range(1, len(spike_times)):
    if isi[i-1] < burst_threshold:
        current_burst.append(i)
    else:
        if len(current_burst) > 1:
            bursts.append(current_burst)
        current_burst = [i]
if len(current_burst) > 1:
    bursts.append(current_burst)

print(f"Detected {len(bursts)} bursts")

isi_per_spike_burst = np.zeros(len(spike_times))
for burst in bursts:
    for i, idx in enumerate(burst):
        isi_per_spike_burst[idx] = 0 if i==0 else (spike_times[idx]-spike_times[burst[i-1]])*1000

In [ ]:
square_wave_bursts, parabolic_bursts, other_bursts = [], [], []
burst_types = {}

for i, burst in enumerate(bursts):
    burst_mask = (time >= spike_times[burst[0]]) & (time <= spike_times[burst[-1]])
    burst_min = np.min(signal[burst_mask])
    prev_mean = np.mean(signal[(time > spike_times[bursts[i-1][-1]]) & (time < spike_times[burst[0]])]) if i>0 else np.nan
    next_mean = np.mean(signal[(time > spike_times[burst[-1]]) & (time < spike_times[bursts[i+1][0]])]) if i < len(bursts)-1 else np.nan
    inter_mean = np.nanmean([prev_mean, next_mean])
    
    if burst_min > inter_mean:
        square_wave_bursts.append(burst)
        burst_types[tuple(burst)] = "Square Wave"
    elif burst_min < inter_mean:
        parabolic_bursts.append(burst)
        burst_types[tuple(burst)] = "Parabolic"
    else:
        other_bursts.append(burst)
        burst_types[tuple(burst)] = "Other"

In [ ]:
burst_list = []
for idx, burst in enumerate(bursts):
    burst_type = burst_types[tuple(burst)]
    burst_list.append([idx+1, spike_times[burst[0]], spike_times[burst[-1]], burst_type])

df_bursts_all = pd.DataFrame(burst_list, columns=["Burst_Number","Start_Time_s","End_Time_s","Type"])
df_bursts_all.to_csv("burst_basic_info_cell89_all_bursts.csv", index=False)
df_bursts_all.head(10)

In [ ]:
colors_map = {"Square Wave":"blue", "Parabolic":"green", "Other":"orange"}
nodes_list, edges_list = [], []

for b_idx, burst in enumerate(bursts):
    burst_type = burst_types[tuple(burst)]
    x_peaks = np.arange(len(burst))
    y_peaks = isi_per_spike_burst[burst]

    G = nx.Graph()
    G.add_nodes_from(range(len(burst)))
    for a in range(len(x_peaks)):
        for b in range(a+1, len(x_peaks)):
            visible = True
            for c in range(a+1,b):
                y_line = y_peaks[b] + (y_peaks[a]-y_peaks[b])*(x_peaks[b]-x_peaks[c])/(x_peaks[b]-x_peaks[a])
                if y_peaks[c] >= y_line:
                    visible=False
                    break
            if visible:
                G.add_edge(a,b)
                edges_list.append([b_idx+1, burst_type, a, b])

    A = nx.to_numpy_array(G)
    n_dim = min(3,A.shape[0])
    embedding_2d = TruncatedSVD(n_components=2, random_state=42).fit_transform(A)
    embedding_3d = TruncatedSVD(n_components=n_dim, random_state=42).fit_transform(A) if n_dim>=3 else np.zeros((len(burst),3))

    for i in range(len(burst)):
        nodes_list.append([b_idx+1, burst_type, i,
                           embedding_2d[i,0], embedding_2d[i,1],
                           embedding_3d[i,0], embedding_3d[i,1], embedding_3d[i,2],
                           burst[i]])

In [ ]:
df_nodes = pd.DataFrame(nodes_list, columns=["Burst_Number","Type","Node_ID",
                                             "X_2D","Y_2D","X_3D","Y_3D","Z_3D",
                                             "Spike_Global_Index"])
df_edges = pd.DataFrame(edges_list, columns=["Burst_Number","Type","Node1_ID","Node2_ID"])

df_nodes.to_csv("burst_nodes_all.csv", index=False)
df_edges.to_csv("burst_edges_all.csv", index=False)
print("CSV files for nodes and edges saved successfully.")